# A Simple NEAT Implementation
Heavily based on: https://github.com/Code-Bullet/NEAT_Template

Original Paper: http://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf

### Imports

In [11]:
import numpy as np
import copy

### Node Class

In [32]:
class Node():
    def __init__(self, num, layer, func):
        self.num = num
        self.func = func
        self.layer = layer
        self.connections_in_num = 0
        self.connections_out = []
        self.in_values = []
        self.done = False
        self.value = 0
    
    def eval(self, value, connection):
        # Checking if the connection is valid
        if connection.out_node != self:
            debug_print("{} not connected to Node {}".format(connection, self.num))
            return None
        
        self.in_values.append(value)
        
        # does not check if a connection submited something twice, so don't run the net agsin until it is done
        if len(self.in_values) != self.connections_in_num:
            self.done = False
            debug_print("Node: {} value recieve: {}, not complete".format(self.num, value))
        else:
            self.value = self.func(sum(self.in_values))
            debug_print("Node: {} value recieve: {}, complete".format(self.num, value))
            for connection in self.connections_out:
                connection.eval(self.value, self)
            self.done = True
            
    
    def reset(self):
        self.done = False
        self.in_values = []
        self.value = 0

### Connection Class
Connects two nodes, when it recives a value from one node it multiplies it by its weight and then evaluates the out node with that value

In [33]:
class Connection():
    def __init__(self, in_node, out_node, w, inno):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = w
        self.innovation_num = inno
        self.enabled = True
    
    def mutate_weight(self, epsilon = .1):
        if np.random.random() < epsilon:
            self.weight = random()
        else:
            self.weight += np.random.normal()
        
        if self.weight > 1:
            self.weight = 1
        elif self.weight < -1:
            self.weight = -1
            
    def eval(self, value, node):
        if self.in_node != node:
            debug_print("Node {} is not connected to connection {}".format(node.num, self))
            return None
        elif self.enabled:
            self.out_node.eval(value*self.weight, self)
        else:
            return False

### The Actual Neural Net Class
This uses the Connection and Node class to create an actual Neural Net that can be Evaluated and Mutated etc.

In [34]:
class Net():
    def __init__(self, num_in, num_out):
        self.in_nodes = []
        self.out_nodes = []
        self.nodes = []
        
        self.connections = []
        
        self.functions = [function]
        self.fitness = 0
        
        self.next_node_num = 0
        self.next_connection_num = 0
        
        # in nodes
        for i in range(num_in):
            self.in_nodes.append(Node(self.next_node_num, 0, function))
            self.nodes.append(self.in_nodes[-1])
            self.next_node_num += 1
        
        self.bias_node = Node(self.next_node_num, 0, bias)
        self.nodes.append(self.bias_node)
        self.next_node_num += 1
        
        # out nodes
        for i in range(num_out):
            self.out_nodes.append(Node(self.next_node_num, 2, function))
            self.nodes.append(self.out_nodes[-1])
            self.next_node_num += 1
        
        # connecting all in nodes to out nodes
        for in_node in self.in_nodes:
            for out_node in self.out_nodes:
                self.connections.append(Connection(in_node, out_node, random(), self.next_connection_num))
                self.next_connection_num += 1
        
        #connecting bias node to out nodes
        for out_node in self.out_nodes:
            self.connections.append(Connection(self.bias_node, out_node, random(), self.next_connection_num))
            self.next_connection_num += 1
        
        # adding connections to nodes
        self.connect()
        
        #setting up input nodes to work
        for node in self.in_nodes:
            node.connections_in_num = 1
            
        self.bias_node.connections_in_num = 1
        
    def eval(self, in_values):
        
        out_values = []
        
        #inserting input values
        for i in range(len(self.in_nodes)):
            self.in_nodes[i].eval(in_values[i], Connection(None, self.in_nodes[i], 1, -1))
        
        self.bias_node.eval(0,  Connection(None, self.bias_node, 1, -1))
        
        #making sure each node is done
        for node in self.nodes:
            while not node.done:
                pass
            debug_print("Node: {} done".format(node.num))
        
        #getting output values
        for node in self.out_nodes:
            out_values.append(node.value)
        
        self.reset()
        
        return out_values
    
    def reset(self):
        # Reset all nodes
        for node in self.nodes:
            node.reset()
    
    def connect(self):
        # Reset all connections of each node
        
        for node in self.nodes:
            node.connections_out = []
            node.connections_in_num = 0
        
        for connection in self.connections:
            if connection.enabled:
                connection.in_node.connections_out.append(connection)
                connection.out_node.connections_in_num += 1
                
        for node in self.in_nodes:
            node.connections_in_num = 1
            
        self.bias_node.connections_in_num = 1
            
    def add_node(self, innovation_history):
        
        connection_old = np.random.choice(self.connections)
        while connection_old.in_node == self.bias_node:
            connection_old = np.random.choice(self.connections)
        
        connection_old.enabled = False
        
        new_node = Node(self.next_node_num, 1, np.random.choice(self.functions))
        self.next_node_num += 1
        self.nodes.append(new_node)
        debug_print("added node {}, between node {} and node {}".format(new_node.num, connection_old.in_node.num, connection_old.out_node.num))
        inno_num = -1
        # connection between old in node and new node
        connection = Connection(connection_old.in_node, new_node, 1, inno_num)
        connection.inovation_num = self.new_innovation_num(innovation_history, connection)
        self.connections.append(connection)
        debug_print("connected node {} to node {}".format(connection.in_node.num, connection.out_node.num))
        # connection between new node and old out node
        connection = Connection(new_node, connection_old.out_node, connection_old.weight, inno_num)
        connection.innovation_num = self.new_innovation_num(innovation_history, connection) # for this function the current inno num of the connection does not matter
        self.connections.append(connection)
        debug_print("connected node {} to node {}".format(connection.in_node.num, connection.out_node.num))
        # connection between bias node and new node
        connection = Connection(self.bias_node, new_node, 0, inno_num)
        connection.inovation_num = self.new_innovation_num(innovation_history, connection)
        self.connections.append(connection)
        debug_print("connected node {} to node {}".format(connection.in_node.num, connection.out_node.num))
        
        self.connect() # reconect everything so it works properly
        
    def add_connection(self, innovation_history):
        if self.fully_connected():
            debug_print("Net {} fully connected, connection failed".format(self))
            return
        
        # looking for node pairs that are viable to connect. Not same node, not to an input node, not from an output node, not connected via another path to avoid circular calculations
        node_pairs = []
        for node_1 in (self.nodes):
            for node_2 in self.nodes:
                if (not (node_1 == node_2)) and (not (node_2.layer == 0)) and (not (node_1.layer == 2)) and (not self.connected(node_2, node_1)) and (not self.connected_directly(node_1, node_2)):
                    debug_print("Viable connection: {} to {}".format(node_1.num, node_2.num))
                    node_pairs.append([node_1, node_2])
        
        if not (len(node_pairs) > 0):
            debug_print("No non circular connections in net: {}".format(self))
            return
        # picking random pair
        idx = np.random.randint(len(node_pairs))
        node_pair = node_pairs[idx]
        # making connection
        connection = Connection(node_pair[0], node_pair[1], random(), -1)
        connection.inovation_num = self.new_innovation_num(innovation_history, connection)
        self.connections.append(connection)
        
        debug_print("made connection between Node {} and Node {}".format(node_pair[0].num, node_pair[1].num))
        
        self.connect() # reconnect everything so every node is properly connected
        
    def new_innovation_num(self, innovation_history, connection):
        # Checking if the innovation already exists, and gets the number
        for innovation in innovation_history[1:]: # first item in list is the global innovation number
            if (innovation[0] == connection.in_node.num) and (innovation[1] == connection.out_node.num):
                return innovation[2]
        
        # If not already an innovation add it to the history
        innovation_num = copy.copy(innovation_history[0]) 
        innovation_history.append([connection.in_node.num, connection.out_node.num, innovation_num])
        innovation_history[0] += 1
        return innovation_num
    
    def fully_connected(self):
        # Checks if the Net is fully connected
        for node in self.nodes:
            if (node.layer == 0) and (len(node.connections_out) < ((len(self.nodes) - len(self.in_nodes)) - 1)):
                debug_print("Node {} on layer {} not fully connected".format(node.num, node.layer))
                return False
            elif (node.layer == 2) and (node.connections_in_num < (len(self.nodes) - len(self.out_nodes))):
                debug_print("Node {} on layer {} not fully connected".format(node.num, node.layer))
                return False
            elif (node.layer == 1) and (node.connections_in_num + len(node.connections_out)) < len(self.nodes):
                debug_print("Node {} on layer {} not fully connected".format(node.num, node.layer))
                return False
        
        return True
    
    def connected(self, node_1, node_2):
        # checks if the two given nodes are connected in that direction via any series of connections
        debug_print("Called, is Node {} connected to Node {}".format(node_1.num, node_2.num))
        for connection in node_1.connections_out:
            if connection.out_node == node_2:
                debug_print("Connected!")
                return True
            elif connection.out_node in self.out_nodes:
                debug_print("Dead end")
            elif self.connected(connection.out_node, node_2):
                debug_print("Connected recieved")
                return True
        debug_print("Not Connected")
        return False
    
    def connected_directly(self, node_1, node_2):
        # checks if two nodes are connected directly.
        for connection in node_1.connections_out:
            if connection.out_node == node_2:
                return True
        
        return False
    
    def Mutate(innovation_history, mutation_rate = [0.8, 0.05, 0.03]):
        # randomly mutates the net, mutations are based on example and rates given in paper
        rand = np.random.random()
        if rand < mutation_rate[0]:
            for connection in self.connections:
                connection.mutate_weight()
                
        rand = np.random.random()
        if rand < mutation_rate[1]:
            self.add_connection(innovation_history)
        
        rand = np.random.random()
        if rand < mutation_rate[2]:
            self.add_node(innovation_history)
        
    def get_innovation_nums(self):
        innovation_nums = []
        for connection in self.connection:
            innovation_nums.append(connection.innovation_num)
            
    def return_by_innovation(self, innovation_num):
        for connection in self.connections:
            if connection.innovation_num == innovation_num:
                return connection
            
        return None



## Trainer

Trains a set of neurasl networks

In [22]:
class Trainer():
    def __init__(self, pop, in_num, out_num):
        self.pop = pop
        self.nets = []
        
        for i in range(pop):
            self.nets.append(Net(in_num, out_num))
        
        self.species = speciate(self.nets)
            
    def run_individual(game):
        for net in nets:
            game.reset()
            while not game.is_dead():
                actions = net.eval(game.observe)
                action = actions.index(max(actions))
                game.act(action)
            net.fitness = game.get_fitness()
    
    def natural_selection(self):
        self.species = speciate(self.nets)
        self.adjust_fitness()
        self.allocate_offspring(self.pop)
        self.kill()
        children = self.create_offspring
        self.nets = children
        
    def kill(self):
        #kill the bottom half of each species
        for s in self.species:
            if s[-1] < 1: # is not allocated any offspring, so it dies
                self.species.remove(s)
            else:
                fitness = [n.fitness for s[:-1] in n]
                idx_sort = np.argsort(fitness)
                keep_idx = idx_sort(round(len(idx_sort)):)
                keep = [s[idx] for idx in keep_idx]
                s = keep + [s[-1]]
        
    def create_offspring(self):
        children = []
        for s in self.species:
            for i in range(s[-1]):
                children.append(get_child(s[:-1]))
        
        return children
    
    # Fitness sharing
    def adjust_fitness(self):
        for species in self.species:
            size = len(species)
            for net in species:
                net.fitness = net.fitness/size
                
    def allocate_offspring(self, num_offspring):
        total_fitness = 0
        for net in nets:
            total_fitness += net.fitness
        
        for species in self.species:
            species_fitness = 0
            for net in species:
                species_fitness += net.fitness
            species.append(round(num_offspring * (species_fitness/total_fitness)))
    
        

SyntaxError: invalid syntax (<ipython-input-22-bec26de2e01c>, line 12)

## K-mediods clustering PAM

Paper Used: https://www.cs.umb.edu/cs738/pam1.pdf

In [20]:
# several ways to improve efficiency?

def cluster(objects, num_clusters, dist_func):
    # innitialize Mediods (S)
    best_mediod = objects[0]
    best_sum = 0
    for obj in objects:
        best_sum += dist_func(best_mediod, obj)
    for obj_1 in objects:
        sum = 0
        for obj_2 in objects:
            sum += dist_func(obj_1, obj_2)
        if sum < best_sum:
            best_sum = sum
            best_mediod = obj_1
    
    objects.remove(best_mediod)
    mediods = [best_mediod]
    
    clusters = [[]]
    # Populating Mediods
    for i in range(num_clusters - 1):
        best_mediod = objects[0]
        best_sum = 0
        for obj_1 in objects:
            sum = 0
            for obj_2 in objects:
                if obj_2 != obj_1:
                    sum += max(closest_dist(obj, mediods, dist_func) - dist_func(obj, best_mediod), 0)
            if sum > best_sum:
                best_sum = sum
                best_mediod = obj
                
        objects.remove(best_mediod)
        mediods.append(best_mediod)
        clusters.append([])
    
    #debug_print("starting Mediods: {}".format(mediods))
    # SWAP Part
    done = False
    while not done:
        min_swap_cost = 0
        for mediod in mediods:
            for obj_1 in objects:
                swap_cost = 0
                for obj_2 in objects:
                    if obj_2 != obj_1:
                        if dist_func(obj_2, mediod) > closest_dist(obj_2, mediods, dist_func): # Case 1
                            contrib = min(dist_func(obj_2, obj_1) - closest_dist(obj_2, mediods, dist_func), 0)
                        else: 
                            contrib = min(dist_func(obj_2, obj_1), second_closest_dist(obj_2, mediods, dist_func)) - closest_dist(obj_2, mediods, dist_func)
                        swap_cost += contrib
                if swap_cost < min_swap_cost:
                    best_pair = [mediod, obj_1]
                    min_swap_cost = swap_cost
        if min_swap_cost < 0:
            mediods.remove(best_pair[0])
            objects.append(best_pair[0])
            objects.remove(best_pair[1])
            mediods.append(best_pair[1])
            #debug_print("swaping {} and {}".format(best_pair[0], best_pair[1]))
        else:
            done = True
            #debug_print("done")
    
    for i in range(len(clusters)):
        clusters[i].append(mediods[i])
    for obj in objects:
        mediod = closest(obj, mediods, dist_func)
        for cluster in clusters:
            if mediod in cluster:
                cluster.append(obj)
    return clusters
    
                            
                
def closest(obj, objects, dist_func):
    best_obj = objects[0]
    best_dist = dist_func(obj, best_obj)
    for obj_2 in objects:
        dist = dist_func(obj, obj_2)
        if dist < best_dist:
            best_dist = dist
            best_obj = obj_2
    
    return best_obj

def closest_dist(obj, objects, dist_func):
    return dist_func(obj, closest(obj, objects, dist_func))
    

def second_closest_dist(obj, objects, dist_func):
    objects_local = copy.deepcopy(objects)
    objects_local.remove(closest(obj, objects_local, dist_func))
    return dist_func(obj, closest(obj, objects_local, dist_func))

# Testing
def dist(a, b):
    x = a[0] - b[0]
    y = a[1] - b[1]
    return (x**2 + y**2)**.5

a = [[0, 0], [0, 1], [1, 2], [2, 1], [1, 1], [1, 0], [0, 2], [2, 0], [9, 9], [10, 10], [9, 10], [10, 9]]

cluster(a, 2,  dist)

[[[1, 1], [0, 1], [1, 2], [2, 1], [1, 0], [0, 2], [2, 0], [0, 0]],
 [[9, 9], [10, 10], [9, 10], [10, 9]]]

### General Functions
Functions that are used elswhere to make it easier and the crossover function that "mates" two Nets based on the method in the paper.

In [36]:
def random():
    return (np.random.random() - 0.5) * 2

def function(x):
    return x

def bias(x):
    return 1

def crossover(net_1, net_2): # net_1 is the more fit parent
    child = Net(len(net_1.in_nodes), len(net_1.out_nodes))
    #the child will have the same input and output structure as net_1
    child.connections = []
    
    for connection_1 in net_1.connections:
        # selecting connection to copy over
        connection_2 = net_2.return_by_innovation(connection_1.innovation_num)
        if connection_2 != None:
            if np.random.random() < 0.5:
                connection = connection_1
            else:
                connection = connection_2
        else:
            connection = connection_1
        
        # making nodes to add to connection and child
        is_enabled = True
        in_node = None
        out_node = None
        
        if (not (connection.enabled)) and np.random.random() < .75:
            is_enabled = False
        
        for node in child.nodes:
            if node.num == connection.in_node.num:
                in_node = node
        
        if not in_node:
            in_node = Node(copy.copy(connection.in_node.num), 1, connection.in_node.func)
            child.nodes.append(in_node)
        
        for node in child.nodes:
            if node.num == connection.out_node.num:
                out_node = node
        
        if not out_node:
            out_node = Node(copy.copy(connection.out_node.num), 1, connection.out_node.func)
            child.nodes.append(out_node)
        
        # Making Connection
        connection = Connection(in_node, out_node, copy.copy(connection.weight), copy.copy(connection.innovation_num))
        connection.enabled = copy.copy(is_enabled)
        child.connections.append(connection)
    
    child.connect()
    
    return child

def net_dist(net_1, net_2, weights = [1, 0.4]): # weights based on those used in paper
    normalizer = max(len(net_1.connections), len(net_2.connections)) - 20
    if normalizer < 1:
        normalizer = 1
    return (weights[0] * num_disjoint_excess(net_1, net_2) / normalizer) + (weights[1] * average_weight_diff(net_1, net_2))
    
def average_weight_diff(net_1, net_2):
    total_diff = 0
    count = 0
    for connection_1 in net_1.connection:
        connection_2 = net_2.return_by_innovation(connection_1.innovation_num)
        if connection_2:
            count += 1
            total_diff += abs(connection_1.weight - connection_2.weight)
    
    if count = 0:
        return 0
    
    return total_diff/count

def num_disjoint_excess(net_1, net_2):
    count = 0
    for connection_1 in net_1.connection:
        connection_2 = net_2.return_by_innovation(connection_1.innovation_num)
        if connection_2:
            count += 1
    
    return (len(net_1.connections) + len(net_2.connections) - 2*count)

def speciate(nets, threshold = 3):
    # first use method outlined in paper to find the # of species we want
    species = []
    for net in nets:
        found = False
        for s in species:
            if (not found) and (net_dist(net, s[0]) < threshold):
                s.append(net)
                found = True
                break
        if not found:
            species.append([net])
    
    # then use that number to cluster the nets
    return cluster(nets, len(species), net_dist)

def select_net_weighted(nets):
    total_fitness = 0
    for net in nets:
        total_fitness += net
    
    rand = np.random.randint(total_fitness)
    
    moving_total = 0
    for net in nets:
        moving_total += net.fitness
        if moving_total > rand:
            return net
        
def get_child(species):
    if np.random.random < .25:
        child = select_net_weighted(species)
    else:
        parent_1 = select_net_weighted(species)
        parent_2 = select_net_weighted(species)
        
        if parent_1.fitness > parent_2.fitness:
            child = crossover(parent_1, parent_2)
        else:
            child = crossover(parent_2, parent_1)
    
    child.mutate()
    return child

def debug_print(var):
    global debug
    if debug:
        print (var)
    

### Testing

In [23]:
debug = True

net_1 = Net(2, 2)
net_2 = Net(2, 2)
history = [5]

print (net_1.eval([1, 2]))

net_1.add_node(history)
net_1.add_connection(history)

net_3 = crossover(net_1, net_2)

print (net_1.eval([1, 2]))
print (net_2.eval([1, 2]))
print (net_3.eval([1, 2]))

NameError: name 'Net' is not defined

In [24]:
a = [1, 2, 3, 4]
a[:-1]

[1, 2, 3]